# Togetherflow: Partial Pooling
**Emergent agent motion dynamics in immersive rooms**

In this notebook, we implement Togetherflow, a computational cognitive model that characterizes the motion pattern of human agents in immersive rooms.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import seaborn as sns


from scipy import stats
from IPython.display import HTML
from numba import njit
from functools import partial

np.set_printoptions(suppress=True)

In [4]:
import tensorflow as tf
import bayesflow as bf

# from bayesflow.simulation import Prior, Simulator, GenerativeModel
from bayesflow.simulation import TwoLevelPrior, Simulator, TwoLevelGenerativeModel

from functools import partial

In [7]:
from priors import (
    partial_pooling_hyper_prior, 
    partial_pooling_local_prior, 
    partial_pooling_shared_prior
)
from utils import count_neighbors


## Generative Model Definition

The movement of any agent $a = 1, ..., A$ is both related to: 1) its interaction with surrounding neighbors $i = 1, ..., I$, which we call *internal influence*, and 2) their motivation to the surrounding spatial objects $j = 1, ..., J$, which we call *external influence*. These influences are modulated by a stationary weight, $w_a$:

\begin{equation}
    \theta_{a, t} = w_a \theta_{a|j, t} + (1 - w_a) \theta_{a|i, t}.
\end{equation}

### Meta-Variables

First, we define some meta-variables, such as the number of agents to simulate, the number of spatial beacons present in the environment, etc.

In [12]:
num_agents = 12
num_beacons = 2
room_size = (8., 10.)
world_size = 25.

## Hyperpriors and priors

In [13]:
test_hypers = partial_pooling_hyper_prior()

In [14]:
test_locals = partial_pooling_local_prior(test_hypers)

In [15]:
prior = TwoLevelPrior(
    hyper_prior_fun=partial_pooling_hyper_prior, 
    local_prior_fun=partial(partial_pooling_local_prior, num_agents=num_agents),
    shared_prior_fun=partial_pooling_shared_prior
)

prior_sim = prior(batch_size=1)
print(prior_sim['shared_parameters'].shape)
print(prior_sim['local_parameters'].shape)
print(prior_sim['hyper_parameters'].shape)

(1, 2)
(1, 12)
(1, 2)


In [16]:
theta = (prior_sim['local_parameters'], prior_sim['shared_parameters'])
theta[1].shape

(1, 2)

### Agent Initialization
First, we initialize the agents with a randomized position and orientation, both uniformly distributed.

In [17]:
@njit
def initialize_agents(
    num_agents: int = 12, 
    room_size: tuple = (8., 10.),
):
    """
    Generate random positions and orientations for agents.

    Parameters
    ----------
    num_agents : int, optional
        Number of agents to generate (default is 100).
    room_size : float, optional
        The size of the boundary within which positions are generated (default is 100.0).

    Returns
    -------
    tuple of np.ndarray
        A tuple containing the positions (np.ndarray) and orientations (np.ndarray) of the agents.
    """
    
    # Generate random positions within the boundary size centered at 0
    x = (np.random.random(size=num_agents).astype(np.float32) - 0.5) * room_size[0]
    y = (np.random.random(size=num_agents).astype(np.float32) - 0.5) * room_size[1]
    positions = np.vstack((x, y)).T
    
    # Generate random orientations (angles in radians between 0 and 2*pi)
    rotations = np.random.random(size=(num_agents, )).astype(np.float32) * np.pi * 2
    
    return positions.astype(np.float32), rotations.astype(np.float32)

### Beacon initialization

To intrinsically motivate the agents, we need a set of virtual beacons that are populated within the environment. The beacons have a freer representation with only positions needed.

In [18]:
@njit
def initialize_beacons(
        num_beacons = 10,
        room_sensing_range = 50.
):
    
    """
    Initialize beacons following a uniform distribution scaled to the room's sensing boundary
    
    Parameters
    ----------
    num_beacons : int, default: 10
        Number of beacons to initialize.
    room_sensing_range : float, default: 50.0
        Size of the environment for the generation of beacons.
    
    Returns
    -------
    beacons      : np.ndarray of shape (num_beacons, 2)
        Initial positions of the beacons. 
    """
    
    beacons = (np.random.random(size=(num_beacons, 2)) - 0.5) * room_sensing_range
    return beacons.astype(np.float32)

## External Influence: drift-diffusion vector

We want to compute the influence of agent movement direction within a single time step. For this, we specify our internal influence as a 2D drift diffusion model, where the agents are approach a spatial beacon within the room's boundary by reorienting its locomotive direction.

\begin{equation}
    \theta_{a|j, t} = \theta_{a|j, t-1} + \omega_a \mathrm{d}t + \mathrm{d}\phi_t,
\end{equation}

\begin{align}
    \mathrm{d}\mathbf{x}_{a|j, t} 
    &= v_{a|j}\mathrm{d}t \frac{\mathbf{x}_{a|j}}{||\mathbf{x}_{a|j}||} + \sigma_{a|j} \mathrm{d}\mathrm{W}_t \\
    &= v_{a}\mathrm{d}t     
    \begin{bmatrix}
        \cos \theta_{a|j, t} \\
        \sin \theta_{a|j, t}
    \end{bmatrix} + \sigma_{a|j} \mathrm{d}\mathrm{W}_t,% \sqrt{\mathrm{d}t} Z_t.
\end{align}

In [19]:
@njit
def external_influence( 
    agent_position,
    target_position,
    noise = 0.1
):
    """
    Generate a drift-diffusion vector in 2D space for a single agent 
    based on a target location (in this case, the position of a beacon).

    Parameters
    ----------
    agent_position : np.ndarray
        The position of the agent.
    target_position : np.ndarray
        The position of the target beacon.
    noise : float, optional
        The rate of diffusion, which determines the variability of the direction (default is 0.1).

    Returns
    -------
    np.ndarray
        A 2D vector representing the drift-diffusion process towards the target (beacon).
    """
    # Calculate the angle towards the beacon (in radian)
    target_angle = np.arctan2(
        target_position[1] - agent_position[1], 
        target_position[0] - agent_position[0]
    )
    
    # Generate a random direction with drift around the target angle
    direction = np.random.vonmises(mu=target_angle, kappa=noise)
    
    # Convert the angle to a unit vector in 2D space
    v = np.array([np.cos(direction), np.sin(direction)], dtype=np.float32)
    
    return v

## Internal Influence: particle dynamics

Its influence by a collective group of agents is modeled as a self-propelling particle system, as expressed in the Vicsek model:

\begin{align}
    \theta_{a|i, t} &= \langle \theta_{i, t}\rangle_{|\mathbf{x}_a - \mathbf{x}_i| < r_a, i \in I} + \eta_{a,t-1}, \\
    \mathrm{d} \mathbf{x}_{a|i,t} &= v_{a,t} \mathrm{d}t
    \begin{bmatrix}
        \cos \theta_{a|i, t} \\
        \sin \theta_{a|i, t}
    \end{bmatrix},
\end{align}

In [20]:
@njit 
def internal_influence(
        self_position,
        other_positions,
        other_rotations,
        sensing_radius = 1.5,
        noise = 0.1
):
    """
    Generate an influence vector for a single agent 
    based on the angular component of the Vicsek model.

    Parameters
    ----------
    self_position : np.ndarray of shape (2,)
        A 2D vector representing the position of the agent
    neighbor_positions : np.ndarray of shape (2,)
        A 2D vector representing the positions of the neighboring agents.
    neighbor_rotations : np.ndarray of shape (2,)
        A 2D vector representing the rotations of the neighboring agents.
    sensing_radius : float
        The sensing radius within which agents interact with their neighbors.
    noise : float, optional
        The level of noise to add to the average direction (default is 0.1).

    Returns
    -------
    np.ndarray
        A 2D unit vector representing the averaged influence direction with added noise.
    """
      
    neighbor_rotations = []
    
    for i in range(len(other_positions)):
        dx = other_positions[i, 0] - self_position[0]
        dy = other_positions[i, 1] - self_position[1]
        d = (dx ** 2 + dy ** 2) ** 0.5
        
        if d <= sensing_radius and d > 0:
            neighbor_rotations.append(other_rotations[i])
            
    if len(neighbor_rotations) == 0:
        return np.array([0.0, 0.0], dtype=np.float32)
    
    neighbor_rotations = np.array(neighbor_rotations)
    averaged_rotation = np.sum(neighbor_rotations) / len(neighbor_rotations)
    
    noise_rotation = (np.random.random() - 0.5) * 2 * noise
    direction = averaged_rotation + noise_rotation
    
    v = np.array([np.cos(direction), np.sin(direction)], dtype=np.float32) 
    
    return v

## Putting everything together: combined influences

The combined influences allow us to update the agents' positions and rotations together.

In [21]:
@njit
def combined_influences(
    agent_positions: np.ndarray = None, 
    agent_rotations: np.ndarray = None,
    beacon_positions: np.ndarray = None,
    velocity: float = 1.0, 
    sensing_radius: float = 2.5,
    dt: float = 0.1, 
    influence_weight: float | np.ndarray = 0.5,
    external_noise: float = 0.1,
    internal_noise: float = 0.1
):
    """
    Update the positions and orientations of a single agent 
    based on velocity and influence vectors.

    Parameters
    ----------
    agent_positions : np.ndarray
        Current positions of the agents.
    agent_rotations : np.ndarray
        Current orientations of the agents.
    velocity : float, optional
        The speed at which agents move (default is 1.0).
    sensing_radius : float, optional
        The sensing radius within which agents interact with their neighbors.
    dt : float, optional
        The time step for updating positions and orientations (default is 0.1).
    influence_weight : float, optional
        The weight of influence_vector1 in determining new orientations (default is 0.7).

    Returns
    -------
    tuple of np.ndarray
        Updated positions (np.ndarray) and orientations (np.ndarray) of the agents.
    """
    
    assert (len(agent_positions) == len(agent_rotations))
    
    num_agents = agent_positions.shape[0]
    num_beacons = beacon_positions.shape[0]
    
    # Check if the shape of the weight is a float
    if isinstance(influence_weight, float):
        weights = np.array([influence_weight])
        weights = np.broadcast_to(weights, shape=(num_agents, ))
    else:
        weights = influence_weight
    
    # Create new numpy arrays for the updated agent positions and rotations
    new_agent_positions = np.zeros((num_agents, 2))
    new_agent_rotations = np.zeros((num_agents, ))
    num_neighbors = np.zeros((num_agents, ))

    for i in range(num_agents):
        # Generate the ddm vector for the agent based on its closest beacon
        num_neighbors[i] = count_neighbors(agent_positions[i], agent_positions)

        distance_to_beacon = []

        for b in range(num_beacons):
            bx = beacon_positions[b, 0] - agent_positions[i, 0]
            by = beacon_positions[b, 1] - agent_positions[i, 1]
            distance_to_beacon.append((bx * bx + by * by) ** 0.5)

        beacon_id = np.argmin(np.array(distance_to_beacon))

        ddm_vector = external_influence(
            agent_positions[i], 
            beacon_positions[beacon_id],
            noise=external_noise
        )

        # Generate the vicsek vector for the agent based on its neighbors (all agents)
        vicsek_vector = internal_influence(
            self_position=agent_positions[i],
            other_positions=agent_positions,
            other_rotations=agent_rotations,
            sensing_radius=sensing_radius,
            noise=internal_noise
        )

        # Update orientations based on two influence vectors
        ddm_influence = np.arctan2(ddm_vector[1], ddm_vector[0])
        vicsek_influence = np.arctan2(vicsek_vector[1], vicsek_vector[0])

        # Combine influences to update orientations with different weights
        new_agent_rotations[i] = agent_rotations[i] + (weights[i].item() * ddm_influence + (1 - weights[i].item()) * vicsek_influence) * dt

        # Ensure orientations are within the range [0, 2*pi]
        new_agent_rotations[i] = np.mod(new_agent_rotations[i], 2 * np.pi)

        # Update positions based on current orientations
        new_agent_positions[i, 0] = agent_positions[i, 0] + velocity * np.cos(new_agent_rotations[i].item()) * dt
        new_agent_positions[i, 1] = agent_positions[i, 1] + velocity * np.sin(new_agent_rotations[i].item()) * dt

    return new_agent_positions, new_agent_rotations, num_neighbors

In [22]:
agent_positions, agent_rotations = initialize_agents(12, room_size=room_size)
beacon_positions = initialize_beacons(num_beacons=2)
new_agent_positions, new_agent_rotations, num_neighbors = combined_influences(agent_positions, agent_rotations, beacon_positions)

## Simulation Loop
The update allows us to continuously simulate the agents' positions and rotations at a given interval

In [23]:
@njit
def simulator_fun(
    theta = None,
    num_agents: int = 12, 
    num_beacons: int = 1,
    room_size: tuple = (8, 10),
    velocity: float = 1.0, 
    dt: float = 0.001, 
    influence_weight: float | np.ndarray = 0.7, 
    sensing_radius: float = 10.0,
    external_noise: float = 0.1,
    internal_noise: float = 0.1,
    time_horizon: float = 30.
):
    """
    Run the simulation and store the time series of positions and orientations of agents.

    Parameters
    ----------
    num_agents : int, optional
        Number of agents to generate (default is 100).
    num_beacons : int, optional
        Number of beacons to generate (default is 1).
    room_size : float, optional
        The size of the boundary within which positions are generated (default is 100).
    velocity : float, optional
        The speed at which agents move (default is 1.0).
    dt : float, optional
        The time step for the update (default is 0.1).
    influence_weight : float, optional
        The weight for influence_vector1 in determining new orientations (default is 0.7).
    sensing_radius : float, optional
        The sensing radius for the Vicsek model (default is 10.0).
    num_timesteps : int, optional
        The number of steps to simulate (default is 100).

    Returns
    -------
    tuple of np.ndarray
        The time series of positions and orientations of the agents.
    """
    
    
    if theta is not None: # Unpack tuples of local and shared priors
        local_params = theta[0]
        shared_params = theta[1]
        
        weights = local_params.copy()
        sensing_radius = shared_params[0].item()
        velocity = shared_params[1].item()
    else:
        weights = np.array([influence_weight], dtype=np.float32)
        weights = np.broadcast_to(weights, shape=(num_agents, ))
    
    num_timesteps = int(time_horizon / dt)
    
    
    # Initialize positions and orientations
    initial_positions, initial_rotations = initialize_agents(num_agents, room_size=room_size)

    # Initialize arrays to store time series of positions and orientations
    positions = np.zeros((num_timesteps, num_agents, 2))
    rotations = np.zeros((num_timesteps, num_agents, ))
    neighbors = np.zeros((num_timesteps, num_agents, ))
    positions[0] = initial_positions
    rotations[0] = initial_rotations
    
    # Initialize beacons
    beacon_positions = initialize_beacons(num_beacons)

    # Simulation loop
    for t in range(1, num_timesteps):
        ps, rs, num_neighbors = combined_influences(
            agent_positions=positions[t-1], 
            agent_rotations=rotations[t-1], 
            beacon_positions=beacon_positions, 
            velocity=velocity, 
            sensing_radius=sensing_radius, 
            dt=dt, 
            influence_weight=weights,
            external_noise=external_noise,
            internal_noise=internal_noise
        )

        # Store positions and orientations for each time step
        positions[t] = ps
        rotations[t] = rs
        neighbors[t] = num_neighbors

    neighbors[0] = neighbors[1]
    
    rotations = rotations[:,:,np.newaxis]
    neighbors = neighbors[:,:,np.newaxis]

    return np.concatenate((positions, rotations, neighbors), axis=-1)

In [24]:
test_hypers = partial_pooling_hyper_prior()
test_locals = partial_pooling_local_prior(test_hypers)
test_shared = partial_pooling_shared_prior()

test_thetas = (test_locals, test_shared)
test_sim = simulator_fun(test_thetas)

## Partial Pooling Generative Model

In [25]:
prior

In [26]:
simulator = Simulator(simulator_fun=partial(simulator_fun, num_agents=num_agents, time_horizon=90, dt=1e-2))
simulator

In [27]:
model = TwoLevelGenerativeModel(
    prior=prior,
    simulator=simulator,
    simulator_is_batched=False,
    name="TogetherFlowPP"
)

INFO:root:Performing 2 pilot runs with the TogetherFlowPP model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, 12)
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, 9000, 12, 4)
INFO:root:Shape of hyper_prior_draws batch after 2 pilot simulations: (batch_size = 2, 2)
INFO:root:Shape of local_prior_draws batch after 2 pilot simulations: (batch_size = 2, 12)
INFO:root:Shape of shared_prior_draws batch after 2 pilot simulations: (batch_size = 2, 2)
INFO:root:No optional simulation batchable context provided.
INFO:root:No optional simulation non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:No optional prior non-batchable context provided.


In [28]:
model(1)

{'sim_data': array([[[[   2.62356615,   -2.44165468,    2.29324746,    1.        ],
          [   3.75672197,    2.75457859,    1.21315885,    0.        ],
          [   3.75114584,   -2.51404023,    0.31385282,    1.        ],
          ...,
          [  -0.68072772,   -0.76673597,    5.28463507,    1.        ],
          [  -0.61379552,    1.7338711 ,    0.50007111,    1.        ],
          [   0.75561285,   -1.89623773,    6.26734018,    2.        ]],
 
         [[   2.60729216,   -2.42323221,    2.29435127,    1.        ],
          [   3.76521965,    2.77764415,    1.21781212,    0.        ],
          [   3.77452948,   -2.5064616 ,    0.31341737,    1.        ],
          ...,
          [  -0.66727002,   -0.78730587,    5.29174059,    1.        ],
          [  -0.59218044,    1.74557661,    0.49632766,    1.        ],
          [   0.78018517,   -1.89689457,    6.25646062,    2.        ]],
 
         [[   2.59087231,   -2.40493961,    2.30229633,    1.        ],
          [   3.

# Configurator

In [48]:
def configurator(input_dict: dict = None, transpose: bool = False):
    
    output_dict = {}
    
    output_dict["hyper_parameters"] = input_dict["hyper_prior_draws"].astype(np.float32, copy=False)
    output_dict["local_parameters"] = input_dict["local_prior_draws"].astype(np.float32, copy=False)
    output_dict["shared_parameters"] = input_dict["shared_prior_draws"].astype(np.float32, copy=False)
    
    output_dict['sim_data'] = input_dict['sim_data']
    
    x = input_dict['sim_data'] / 10. 
    
    if transpose:
        x = np.moveaxis(x, 2, 1)[:, :, ::10, :]
    
    output_dict['summary_conditions'] = x.astype(np.float32, copy=False)
    
    return output_dict

In [30]:
sim = model(5)
out = configurator(sim)
print(out['hyper_parameters'].shape)
print(out['local_parameters'].shape)
print(out['shared_parameters'].shape)
print(out['summary_conditions'].shape)

(5, 2)
(5, 12)
(5, 2)
(5, 12, 900, 4)


In [31]:
np.concatenate((out['local_parameters'], out['shared_parameters']), axis=-1)

array([[0.7835725 , 0.267219  , 0.31101918, 0.6999683 , 0.5075343 ,
        0.41161084, 0.19561529, 0.51038   , 0.527022  , 0.26017255,
        0.35136452, 0.47310403, 0.40123734, 2.7103755 ],
       [0.34415168, 0.30185863, 0.47507027, 0.36615327, 0.5446215 ,
        0.67031276, 0.56910956, 0.573689  , 0.28751165, 0.31597298,
        0.42138138, 0.6484708 , 0.02313599, 3.0592139 ],
       [0.29899427, 0.6720077 , 0.8153631 , 0.7192794 , 0.5487453 ,
        0.6652825 , 0.4952268 , 0.4251791 , 0.47840667, 0.5615901 ,
        0.79784477, 0.75555074, 0.26337874, 2.3307288 ],
       [0.41206247, 0.26556748, 0.46494582, 0.50865644, 0.22126868,
        0.25341704, 0.11091977, 0.54277825, 0.16025545, 0.10664769,
        0.18157975, 0.3671722 , 0.21219394, 1.4472896 ],
       [0.69776225, 0.7137571 , 0.06229743, 0.6471252 , 0.68015885,
        0.3619292 , 0.49035767, 0.74407846, 0.69239634, 0.5162026 ,
        0.27458283, 0.6048591 , 0.42529514, 2.3249953 ]], dtype=float32)

# Neural Approximator

How many parameters are there? We have:

- Hyperparameters: $\alpha_w$, $\beta_w$;
- Local parameters: $w_a$, one for each agent;
- Shared parameters: $r$, and $v$.

In [50]:
summary_net = bf.summary_networks.HierarchicalNetwork([
    tf.keras.layers.TimeDistributed(tf.keras.layers.LSTM(units=256)),
    bf.networks.SetTransformer(num_inducing_points=None, input_dim=256, summary_dim=16)
])

summaries = summary_net(out["sim_data"])

In [51]:
local_inference_net = bf.inference_networks.InvertibleNetwork(
    num_params=12,
    # num_coupling_layers=6,
    # coupling_design="spline",
    # coupling_settings={
    #     'units': 128,
    #     'activation': 'swish',
    #     'kernel_regularizer': None,
    #     'dropout_prob': 0.0
    # }
)

global_inference_net = bf.inference_networks.InvertibleNetwork(
    num_params=4, 
    # num_coupling_layers=6,
    # coupling_design="spline",
    # coupling_settings={
    #     'units': 128,
    #     'activation': 'swish',
    #     'kernel_regularizer': None,
    #     'dropout_prob': 0.0
    # }
)

local_amortizer = bf.amortizers.AmortizedPosterior(
    inference_net=local_inference_net
)

global_amortizer = bf.amortizers.AmortizedPosterior(
    inference_net=global_inference_net
)

amortizer = bf.amortizers.TwoLevelAmortizedPosterior(
    summary_net=summary_net, 
    local_amortizer=local_amortizer,
    global_amortizer=global_amortizer
)

## Online Training

In [53]:
trainer = bf.trainers.Trainer(
    amortizer=amortizer,
    generative_model=model,
    configurator=configurator,
    checkpoint_path="checkpoints/partial1"
)

INFO:root:Initialized empty loss history.
INFO:root:Initialized networks from scratch.
INFO:root:Performing a consistency check with provided components...


ConfigurationError: Could not carry out computations of generative_model ->configurator -> amortizer -> loss! Error trace:
 Exception encountered when calling layer 'dense_coupling_net_96' (type DenseCouplingNet).

{{function_node __wrapped__ConcatV2_N_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} ConcatOp : Ranks of all input tensors should match: shape[0] = [2,6] vs. shape[1] = [2,9000,260] [Op:ConcatV2] name: concat

Call arguments received by layer 'dense_coupling_net_96' (type DenseCouplingNet):
  • target=tf.Tensor(shape=(2, 6), dtype=float32)
  • condition=tf.Tensor(shape=(2, 9000, 260), dtype=float32)
  • kwargs={'training': 'None'}

In [ ]:
history = trainer.train_online(epochs=3, batch_size=32, iterations_per_epoch=500)

## Diagnostics

In [ ]:
validation_sim = model(400)
validation_configured = configurator(validation_sim)

validation_configured["parameters"]

In [ ]:
post_samples = amortizer.sample(validation_configured, n_samples=1000)
prior_samples = validation_configured["parameters"]

In [ ]:
sns.set(rc={'axes.facecolor':'#FFFFFF00', 'figure.facecolor':'#FFFFFF00'})
sns.set_style('whitegrid')

g = bf.diagnostics.plot_recovery(
    post_samples=post_samples, 
    prior_samples=prior_samples, 
    param_names=param_names,
    color="#4E2A84"
)

In [ ]:
c = bf.diagnostics.plot_z_score_contraction(post_samples=post_samples, prior_samples=prior_samples, param_names=param_names)

In [ ]:
h = bf.diagnostics.plot_sbc_histograms(post_samples=post_samples, prior_samples=prior_samples, param_names=param_names, num_bins=10)

In [ ]:
e = bf.diagnostics.plot_sbc_ecdf(
    post_samples=post_samples, 
    prior_samples=prior_samples, 
    param_names=param_names, 
    difference=True,
    rank_ecdf_color="#4E2A84"    
)

In [ ]:
d = bf.diagnostics.plot_posterior_2d(posterior_draws=post_samples[1], prior_draws=prior_samples, param_names=param_names)